# Accessing an API

We will use https://replicate.com services through an API. Since we are not running AI tools in colab we don't need a GPU runtime.

⚡ You have to login using your github account, it is free to a (unknown) limit.

To use an API you have to provide some kind of verification, and it is mainly done using **tokens**. Tokens are strings of characters attached to your account in the service, you have to keep them "secret". ⚡ So we need to create a replicate token here https://replicate.com/account/api-tokens

⚡ Then you can check the documentation about accessing the API here https://replicate.com/docs/get-started/python.


⚡ Then search for the service that you want to use here https://replicate.com/explore. For this example we are going to use the [Llama language model](https://replicate.com/meta/llama-2-13b-chat/). We can check its API usage in the API tab, selecting python as the programming language ([link](https://replicate.com/meta/llama-2-13b-chat/api?tab=python)).

## Steps

First we install the library for accessing the API.

In [ ]:
!pip install -q replicate datasets

In [ ]:
import replicate

api_token='YOUR_REPLICATE_TOKEN_HERE'

client = replicate.Client(api_token=api_token)

Each model have different parameters. Llama has a system prompt, that defines the role of the LLM and a prompt that is the instruction or example. You can check the parameters on the [API tab](https://replicate.com/meta/llama-2-13b-chat/api).

Some work is required to craft the system prompt and the prompt, and there is a lot of trial an error. There are lot's of online resources online.

Now we will use another LLM, [Mistral-7B-Instruct v0.2](https://replicate.com/mistralai/mistral-7b-instruct-v0.2) to generate image prompts using a complex prompt.

In [ ]:
def create_image_prompt(dream):

  prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  You are an excellent Stable diffusion prompt writer. Given a one-sentence story, you can write a Stable diffusion prompt.
  Instructions for writing a Stable diffusion prompt with a one-sentence story:
  • Use specific descriptors based on the one-sentence story details (e.g., "dense pine forest" instead of "forest").
  • Detail objects mentioned in the one-sentence story (e.g., "ancient leather-bound book" instead of "book").
  • Highlight emotions conveyed in the one-sentence story (e.g., "melancholic," "joyful").
  • Choose a visual style, such as "35mm film," "cinematic," "watercolor", "oil painting", "3D art".
  • Reference an artist or artistic style that aligns with the one-sentence story vibe.

  Examples of Stable diffusion prompts:
  • "Macie Grey in a Victorian dress, exploring a luminous garden with glowing plants and mystical creatures, rendered in a dreamy watercolor style with soft pastel hues."
  • "Mark Hamel as an astronaut in a futuristic suit, examining an ancient monolith amidst ruins on an alien planet, captured with a 35mm film grain effect and a surreal, otherworldly color palette."
  • "Baby shark with a painted face on an old wall, in the style of hyper-realistic sculptures, fragmented figures, distressed materials, tiled walls of light grays, cracked, rococo—inspired art"
  • "Award-winning cinematic bioluminescent oil creature design in gold, vibrant holographic gradient blue and silver colored scheme, in the style 3d hydro — drip venom character, ray tracing reflection, prismatic lighting, realistic texture detail, vibrant electric flames coursing through oil"
  • "Pope Francis wearing leather jacket is a DJ in a nightclub, mixing live on stage, giant mixing table, 4k resolution, a masterpiece"
  • "Kanye West in medieval armor, standing on a cliff's edge, watching a dragon soar from misty mountains, depicted in a Renaissance painting style with dramatic chiaroscuro lighting."
  • "A trench-coated Dick Tracy, silhouette against neon noir streets, searching for elusive clues of a jewel thief, visualized in a vibrant neon noir style with rain-soaked streets reflecting the city's lights."

  Dream description: {dream}

  ### Response:"""

  output = client.run(
      "mistralai/mistral-7b-instruct-v0.2",
      input={
          "prompt": prompt,
          "max_new_tokens": 256,
          "prompt_template": "<s>[INST] {prompt} [/INST] ",
      },
  )

  image_prompt = "".join(output)

  return image_prompt

In [ ]:
from datasets import load_dataset

dataset = load_dataset("DReAMy-lib/DreamBank-dreams-en")

In [ ]:
first_dreams = dataset['train']['dreams'][0:5]

image_prompts = []

for dream in first_dreams:
  print(dream)
  image_prompt = create_image_prompt(dream)
  print(image_prompt)

  image_prompts.append(image_prompt)

In [ ]:
for image_prompt in image_prompts:
  print(image_prompt)

In [ ]:
#@title Clean the image prompt

import re

def clean_prompt(prompt):

  # https://stackoverflow.com/a/49146722/330558
  def remove_emoji(string):
      emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            "]+", flags=re.UNICODE)
      return emoji_pattern.sub(r'', string)

  clean_image_prompt = remove_emoji(image_prompt)
  # text_between_quotes = re.findall('"([^"]*)"', clean_image_prompt)
  # if len(text_between_quotes) > 0:
  #   clean_image_prompt = text_between_quotes[0]
  print(clean_image_prompt)

  return clean_image_prompt

In [ ]:
clean_image_prompts = []
for image_prompt in image_prompts:
  clean_image_prompts = clean_prompt(image_prompt)

In [ ]:
def generate_image(image_prompt):
  output = client.run(
      "stability-ai/sdxl:39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b",
      input={
          "width": 768,
          "height": 768,
          "prompt": image_prompt,
          "num_outputs": 1,
      }
  )

  image_url = output[0]
  image = Image.open(requests.get(image_url, stream=True).raw)

  return image

In [ ]:
import requests
from PIL import Image

for clean_image_prompt in clean_image_prompts:
  image = generate_image(clean_image_prompt)
  display(image)

The output is a list of URLs to images, so we iterate it downloading and displaying the image.

In [ ]:
import requests
from PIL import Image

for url in output:
  image = Image.open(requests.get(url, stream=True).raw)
  display(image)

# Finalizing

When you finish working you have to remember to **stop the runtime**, because there is a time limit and to avoid wasting resources. To stop the runtime click Manage Sessions on the Runtime menu. Once the dialog opens click terminate on the current runtime.

> But when you stop the runtime everything you have not saved is ⚠ **lost** ⚠, so be sure to **download** everything you want to keep before stopping it.


# Credits

Taller Estampa https://tallerestampa.com / https://github.com/estampa
